## Collecting Flights data

In [ ]:
import requests
import json
from IPython.display import JSON
from keys import *

In [16]:

#be careful not to run too often as there is a maximum of 200 api calls per month

url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/EDDB/2025-09-01T08:00/2025-09-01T20:00"

querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}

headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': flights_key
    }

response = requests.request("GET", url, headers=headers, params=querystring)

In [18]:
if response.status_code == 200:
    new_data = response.json()

    try:
        with open("data.json", "r") as json_file:
            existing_data = json.load(json_file)
    except (FileNotFoundError, json.decoder.JSONDecodeError):
        existing_data = []

    existing_data.append(new_data)

    with open("data.json", "w") as json_file:
        json.dump(existing_data, json_file, indent=4)
        print("Data appended to data.json file.")
else:
    print("Failed to retrieve data from the API. Status code:", response.status_code)

Data appended to data.json file.


In [19]:
flight_arrivals = response.json()

for flight in flight_arrivals:
    flight_arrivals['arrivals'][0]['arrival']['scheduledTime']
    flight_arrivals['arrivals'][0]['number']
    flight_arrivals['arrivals'][0]['arrival']['terminal']
    flight_arrivals['arrivals'][0]['departure']['airport']['name']
    flight_arrivals['arrivals'][0]['aircraft']['model']
    flight_arrivals['arrivals'][0]['airline']['name']

In [21]:
import pandas as pd
from numpy import nan

flight_arrivals = response.json()

def flight_extraction(flights):
    flights_df = pd.json_normalize({
        'Scheduled_arrival_time': flights['arrival']['scheduledTime'],
        'flight_number': flights['number'],
        'from' : flights['departure']['airport']['name'],
        'airline' : flights['airline']['name'],
        'aircraft' : flights['aircraft']['model']
    })
    return(flights_df)

flight_arrivals = pd.concat(
    [flight_extraction(flight) for flight in flight_arrivals['arrivals']]
)

flight_arrivals.head()

,flight_number,from,airline,aircraft,Scheduled_arrival_time.utc,Scheduled_arrival_time.local
0,DE 4095,Frankfurt-am-Main,Condor,Airbus A320,2025-09-01 05:55Z,2025-09-01 07:55+02:00
0,DE 2501,Dubai,Condor,Airbus A320 NEO,2025-09-01 06:25Z,2025-09-01 08:25+02:00
0,PC 5067,Antalya,Pegasus,Airbus A320 (Sharklets),2025-09-01 06:35Z,2025-09-01 08:35+02:00
0,AY 1431,Helsinki,Finnair,Airbus A319,2025-09-01 06:05Z,2025-09-01 08:05+02:00
0,JL 6899,Helsinki,JL,Airbus A319,2025-09-01 06:05Z,2025-09-01 08:05+02:00
